In [13]:
import os
from collections import Counter
from string import punctuation
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [10]:
lmtzr = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
stopwords = nltk.corpus.stopwords.words('english')

def processLine(line):
    line = line.lower()
    tokens = tokenizer.tokenize(line)
    output = []
    for t in tokens:
        t = lmtzr.lemmatize(t)
        if (len(t) <= 2) or (len(t) >= 15) or (t in stopwords) or (t.isdigit()):
            continue
        output.append(t)
    return output

In [92]:
manufacture = ['MENTOR', 'ALLERGAN', 'IDEAL', 'SIENTRA', 'MCGHAN', 'INAMED', 'SILIMED', 'NAGOR']
fill_type = ['SALINE', 'SILICONE', 'GEL', 'COHESIVE']
surface_type = ['SMOOTH', 'TEXTURED', 'BIOCELL']
implantation_indication = ['AUGMENTATION', 'RECONSTRUCTION', 'COSMETIC']
ALCL = ['ALCL', 'ANAPLASTIC LARGE CELL LYMPHOMA', 'LYMPHOMA', 'T-CELL LYMPHOMA', 'B-CELL LYMPHOMA']
side = ['LEFT', 'RIGHT']
symptom = ['breast pain', 'breast swelling', 'breast cyst', 'breast calcification', 'capsular contracture',
           'lymph node enlargement', 'firmness of breast', 'hematoma', 'mass', 'lump', 'rupture', 'deflated',
           'infection', 'abscess', 'leukopenia', 'nodules', 'skin discoloration', 'skin lesion', 'seroma',
           'effusion', 'fluid']
word_list = [manufacture, fill_type, surface_type, implantation_indication, ALCL, side, symptom]

MDR_KEY_LIST = []
with open('Data/No time filter/FWM_Key_no_time_filter.txt') as fh:
    lines = fh.readlines()
    for line in lines:
        MDR_KEY_LIST.append(line.rstrip('\n'))
with open('Data/No time filter/FTR_Key_no_time_filter.txt') as fh:
    lines = fh.readlines()
    for line in lines:
        MDR_KEY_LIST.append(line.rstrip('\n'))

In [4]:
processed = []
for l in word_list:
    for w in l:
        p = processLine(w)
        print(w, ':', p)

MENTOR : mentor
ALLERGAN : allergan
IDEAL : ideal
SIENTRA : sientra
MCGHAN : mcghan
INAMED : inamed
SILIMED : silimed
NAGOR : nagor
SALINE : saline
SILICONE : silicone
COHESIVE : cohesive
SMOOTH : smooth
TEXTURED : textured
BIOCELL : biocell
AUGMENTATION : augmentation
RECONSTRUCTION : reconstruction
COSMETIC : cosmetic
ANAPLASTIC LARGE CELL LYMPHOMA : anaplastic large cell lymphoma
LYMPHOMA : lymphoma
T-CELL LYMPHOMA : cell lymphoma
B-CELL LYMPHOMA : cell lymphoma
left : left
right : right
breast pain : breast pain
breast swelling : breast swelling
breast cyst : breast cyst
breast calcification : breast calcification
capsular contracture : capsular contracture
lymph node enlargement : lymph node enlargement
firmness of breasthematoma : firmness breasthematoma
mass : mass
lump : lump
rupture : rupture
deflated : deflated
infection : infection
abscess : abscess
leukopenia : leukopenia
nodules : nodule
skin discoloration : skin discoloration
skin lesion : skin lesion
seroma : seroma
effu

In [26]:
def filtering(s):
    if (s == 'FWM') | (s == 'FTR'):
        return True
    else:
        return False

In [27]:
FOI_DEV_LIST = ['', 'Add', 'Change', 'thru1997']
for i in range(1998, 2017):
    FOI_DEV_LIST.append(str(i))

df_list = []
for s in FOI_DEV_LIST:
    df = pd.read_csv('foidev/foidev'+s+'.txt', sep='|', header=0, encoding='ISO-8859-1', error_bad_lines=False)
    df_list.append(df)
df_BASELINE = pd.concat(df_list, axis=0)
del df_list

df_BASELINE['filter'] = df_BASELINE['DEVICE_REPORT_PRODUCT_CODE'].map(filtering)
df_BI_DEV = df_BASELINE.loc[df_BASELINE['filter']==True, :]

b'Skipping line 3974: expected 28 fields, saw 29\n'
b'Skipping line 46727: expected 28 fields, saw 29\n'
b'Skipping line 7949: expected 28 fields, saw 29\n'
b'Skipping line 24283: expected 28 fields, saw 29\n'
b'Skipping line 54015: expected 45 fields, saw 47\n'
b'Skipping line 66558: expected 45 fields, saw 58\n'
b'Skipping line 121357: expected 45 fields, saw 59\nSkipping line 122019: expected 45 fields, saw 59\nSkipping line 129021: expected 45 fields, saw 58\n'
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,12,13,14,16,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14) have mixed types. Specify dtype optio

In [35]:
print(df_BI_DEV.shape[0])
# del df_BI_DEV['filter']
for n in df_BI_DEV.columns.values:
    if 'BASELINE' in n:
        del df_BI_DEV[n]
df_BI_DEV.to_csv('WF/DEV_BI_FULL_TABLE.txt', header=True, index=False, sep='|')

29031


In [42]:
print('Manufacturer word frequecy:\n')
for m in manufacture:
    counter = 0
    fh = open('WF/manufacturer/'+m+'.txt', 'w')
    for index, value in df_BI_DEV.iterrows():
        l = str(value['MANUFACTURER_D_NAME']).split(' ')
        if m in l:
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Manufacturer word frequecy:

MENTOR : 3439
ALLERGAN : 2100
IDEAL : 24
SIENTRA : 23
MCGHAN : 2120
INAMED : 189
SILIMED : 7
NAGOR : 4


In [62]:
FOI_TEXT_LIST = ['', 'Add', 'Change', 'thru1995']
for i in range(1996, 2017):
    FOI_TEXT_LIST.append(str(i))

df_BI = pd.DataFrame(df_BI_DEV['MDR_REPORT_KEY'])
df_BI_TEXT = pd.DataFrame()
for s in FOI_TEXT_LIST:
    df = pd.read_csv('foitext/foitext'+s+'.txt', sep='|', header=0, encoding='ISO-8859-1', error_bad_lines=False)
    temp = df.merge(df_BI, on=['MDR_REPORT_KEY'], how='inner')
    df_BI_TEXT = pd.concat([df_BI_TEXT, temp], axis=0)
    del temp

print(df_BI_TEXT.shape[0])
df_BI_TEXT.head()

b'Skipping line 12326: expected 6 fields, saw 7\nSkipping line 41095: expected 6 fields, saw 7\nSkipping line 58906: expected 6 fields, saw 14\nSkipping line 105443: expected 6 fields, saw 8\n'
b'Skipping line 136392: expected 6 fields, saw 7\nSkipping line 175249: expected 6 fields, saw 7\nSkipping line 212357: expected 6 fields, saw 7\nSkipping line 244567: expected 6 fields, saw 7\nSkipping line 252598: expected 6 fields, saw 8\n'
b'Skipping line 278732: expected 6 fields, saw 7\nSkipping line 315087: expected 6 fields, saw 7\nSkipping line 322888: expected 6 fields, saw 7\nSkipping line 374939: expected 6 fields, saw 7\nSkipping line 388628: expected 6 fields, saw 7\n'
b'Skipping line 469829: expected 6 fields, saw 7\nSkipping line 471072: expected 6 fields, saw 7\nSkipping line 508184: expected 6 fields, saw 7\nSkipping line 508372: expected 6 fields, saw 7\n'
b'Skipping line 548966: expected 6 fields, saw 8\nSkipping line 549426: expected 6 fields, saw 7\nSkipping line 574290: ex

b'Skipping line 145016: expected 6 fields, saw 7\nSkipping line 145918: expected 6 fields, saw 15\nSkipping line 152452: expected 6 fields, saw 14\nSkipping line 171616: expected 6 fields, saw 7\nSkipping line 173453: expected 6 fields, saw 7\nSkipping line 175441: expected 6 fields, saw 7\nSkipping line 182287: expected 6 fields, saw 7\nSkipping line 188206: expected 6 fields, saw 7\nSkipping line 204131: expected 6 fields, saw 13\nSkipping line 236093: expected 6 fields, saw 12\nSkipping line 247721: expected 6 fields, saw 8\n'
b'Skipping line 266986: expected 6 fields, saw 8\nSkipping line 280145: expected 6 fields, saw 8\nSkipping line 285217: expected 6 fields, saw 7\nSkipping line 285986: expected 6 fields, saw 10\nSkipping line 289821: expected 6 fields, saw 8\nSkipping line 298732: expected 6 fields, saw 9\nSkipping line 304031: expected 6 fields, saw 7\nSkipping line 317389: expected 6 fields, saw 7\nSkipping line 342015: expected 6 fields, saw 10\nSkipping line 349838: expect

b'Skipping line 2098204: expected 6 fields, saw 7\nSkipping line 2143443: expected 6 fields, saw 10\nSkipping line 2156060: expected 6 fields, saw 9\nSkipping line 2168863: expected 6 fields, saw 13\nSkipping line 2174632: expected 6 fields, saw 10\nSkipping line 2178925: expected 6 fields, saw 19\nSkipping line 2198300: expected 6 fields, saw 7\nSkipping line 2209664: expected 6 fields, saw 13\nSkipping line 2209972: expected 6 fields, saw 7\nSkipping line 2211883: expected 6 fields, saw 8\n'
b'Skipping line 2230573: expected 6 fields, saw 8\nSkipping line 2231276: expected 6 fields, saw 8\nSkipping line 2233511: expected 6 fields, saw 8\nSkipping line 2238958: expected 6 fields, saw 7\nSkipping line 2240449: expected 6 fields, saw 7\nSkipping line 2245124: expected 6 fields, saw 7\nSkipping line 2248925: expected 6 fields, saw 13\nSkipping line 2256102: expected 6 fields, saw 9\n'
b'Skipping line 1427: expected 6 fields, saw 7\nSkipping line 35232: expected 6 fields, saw 8\nSkipping 

34208


,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT
0,6730886,80620016,D,1,NaN,HEALTHCARE PROFESSIONAL REPORTED A LEFT SIDE D...
1,6730886,80620016,D,1,NaN,HEALTHCARE PROFESSIONAL REPORTED A LEFT SIDE D...
2,6730886,80620016,D,1,NaN,HEALTHCARE PROFESSIONAL REPORTED A LEFT SIDE D...
3,6730886,80620015,N,1,NaN,DEVICE EVALUATION: VISUAL ANALYSIS OF THE RETU...
4,6730886,80620015,N,1,NaN,DEVICE EVALUATION: VISUAL ANALYSIS OF THE RETU...


In [63]:
df_BI_TEXT = df_BI_TEXT.drop_duplicates('MDR_TEXT_KEY')
print(df_BI_TEXT.shape[0])
df_BI_TEXT.to_csv('WF/TEXT_FULL_BI_LIST.txt', header=True, index=False, sep='|')
df_BI_TEXT.head()

27137


,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT
0,6730886,80620016,D,1,NaN,HEALTHCARE PROFESSIONAL REPORTED A LEFT SIDE D...
3,6730886,80620015,N,1,NaN,DEVICE EVALUATION: VISUAL ANALYSIS OF THE RETU...
6,6734192,80750975,N,1,NaN,THE DEVICE REMAINS IMPLANTED. A REVIEW OF THE ...
9,6734192,80750976,D,1,NaN,"PATIENT REPORTED THAT THE LEFT SIDE ""HAS COLLA..."
12,6283766,66229409,D,1,NaN,DEFLATION RESULTING IN EXPLANTATION.


In [71]:
punctuations = '''!()[]{};:'"\,<>./?@#$%^&*_~'''

def processing_text(s):
    s = str(s)
    no_punct = ""
    for char in s:
        if char not in punctuations:
            no_punct = no_punct + char
    l = no_punct.split(' ')
    output = []
    for w in l:
        if (w in stopwords) or (w.isdigit()):
            continue
        output.append(w)
    return ' '.join(output)

In [72]:
df_BI_TEXT['FOI_TEXT'] = df_BI_TEXT['FOI_TEXT'].map(processing_text)
df_BI_TEXT['FOI_TEXT'].head()

0    HEALTHCARE PROFESSIONAL REPORTED A LEFT SIDE D...
1    DEVICE EVALUATION VISUAL ANALYSIS OF THE RETUR...
2    THE DEVICE REMAINS IMPLANTED A REVIEW OF THE D...
3    PATIENT REPORTED THAT THE LEFT SIDE HAS COLLAP...
4                  DEFLATION RESULTING IN EXPLANTATION
Name: FOI_TEXT, dtype: object

In [93]:
print('Fill type word frequecy:\n')
for m in fill_type:
    counter = 0
    fh = open('WF/fill_type/'+m+'.txt', 'w')
    for index, value in df_BI_TEXT.iterrows():
        l = str(value['FOI_TEXT']).split(' ')
        if m in l:
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Fill type word frequecy:

SALINE : 3309
SILICONE : 8184
GEL : 5113
COHESIVE : 74


In [75]:
print('Surface type word frequecy:\n')
for m in surface_type:
    counter = 0
    fh = open('WF/surface_type/'+m+'.txt', 'w')
    for index, value in df_BI_TEXT.iterrows():
        l = str(value['FOI_TEXT']).split(' ')
        if m in l:
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Surface type word frequecy:

SMOOTH : 372
TEXTURED : 244
BIOCELL : 17


In [76]:
print('Implantation indication word frequecy:\n')
for m in implantation_indication:
    counter = 0
    fh = open('WF/implantation_indication/'+m+'.txt', 'w')
    for index, value in df_BI_TEXT.iterrows():
        l = str(value['FOI_TEXT']).split(' ')
        if m in l:
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Implantation indication word frequecy:

AUGMENTATION : 1795
RECONSTRUCTION : 981
COSMETIC : 226


In [84]:
print('ALCL word frequecy:\n')
for m in ALCL:
    counter = 0
    fh = open('WF/ALCL/'+m+'.txt', 'w')
    for index, value in df_BI_TEXT.iterrows():
        if m in str(value['FOI_TEXT']):
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

ALCL word frequecy:

ALCL : 875
ANAPLASTIC LARGE CELL LYMPHOMA : 471
LYMPHOMA : 1033
T-CELL LYMPHOMA : 73
B-CELL LYMPHOMA : 1


In [87]:
print('Side word frequecy:\n')
for m in side:
    counter = 0
    fh = open('WF/side/'+m+'.txt', 'w')
    for index, value in df_BI_TEXT.iterrows():
        if m in str(value['FOI_TEXT']):
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Side word frequecy:

LEFT : 6543
RIGHT : 6609


In [89]:
print('Symptom word frequecy:\n')
for m in symptom:
    m = m.upper()
    counter = 0
    fh = open('WF/symptom/'+m+'.txt', 'w')
    for index, value in df_BI_TEXT.iterrows():
        if m in str(value['FOI_TEXT']):
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Symptom word frequecy:

BREAST PAIN : 706
BREAST SWELLING : 35
BREAST CYST : 5
BREAST CALCIFICATION : 3
CAPSULAR CONTRACTURE : 2979
LYMPH NODE ENLARGEMENT : 3
FIRMNESS OF BREASTHEMATOMA : 0
MASS : 594
LUMP : 674
RUPTURE : 6573
DEFLATED : 1125
INFECTION : 1890
ABSCESS : 45
LEUKOPENIA : 2
NODULES : 76
SKIN DISCOLORATION : 10
SKIN LESION : 17
SEROMA : 1125
EFFUSION : 45
FLUID : 637


In [95]:
df_BI_DEV['BRAND_NAME'] = df_BI_DEV['BRAND_NAME'].map(processing_text)
df_BI_DEV['BRAND_NAME'].head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


887                                       BREAST IMPLANTS
888                                       BREAST IMPLANTS
1705    SALINESILICONE BREAST IMPLANT RIGHT AND LEFT U...
2263                   STYLE SALINE FILLED BREAST IMPLANT
2327                   STYLE SALINE FILLED BREAST IMPLANT
Name: BRAND_NAME, dtype: object

In [101]:
print('Surface type word frequecy (brand name):\n')
for m in surface_type:
    counter = 0
    fh = open('WF/surface_type_BRAND_NAME/'+m+'.txt', 'w')
    for index, value in df_BI_DEV.iterrows():
        if m in str(value['BRAND_NAME']):
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Surface type word frequecy (brand name):

SMOOTH : 808
TEXTURED : 406
BIOCELL : 53


In [102]:
print('Fill type word frequecy (brand name):\n')
for m in fill_type:
    counter = 0
    fh = open('WF/fill_type_BRAND_NAME/'+m+'.txt', 'w')
    for index, value in df_BI_DEV.iterrows():
        if m in str(value['BRAND_NAME']):
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Fill type word frequecy (brand name):

SALINE : 3711
SILICONE : 2325
GEL : 4884
COHESIVE : 160


In [103]:
print('Surface type word frequecy (generic name):\n')
for m in surface_type:
    counter = 0
    fh = open('WF/surface_type_GENERIC_NAME/'+m+'.txt', 'w')
    for index, value in df_BI_DEV.iterrows():
        if m in str(value['GENERIC_NAME']):
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Surface type word frequecy (generic name):

SMOOTH : 164
TEXTURED : 379
BIOCELL : 8


In [104]:
print('Fill type word frequecy (generic name):\n')
for m in fill_type:
    counter = 0
    fh = open('WF/fill_type_GENERIC_NAME/'+m+'.txt', 'w')
    for index, value in df_BI_DEV.iterrows():
        if m in str(value['GENERIC_NAME']):
            fh.write(str(value['MDR_REPORT_KEY']) + '\n')
            counter += 1
    print(m, ':', counter)
    fh.close()

Fill type word frequecy (generic name):

SALINE : 5217
SILICONE : 3704
GEL : 4932
COHESIVE : 28


## Read BI text file again...

In [12]:
df_BI_TEXT = pd.read_csv('WF/TEXT_FULL_BI_LIST.txt', header=0, sep='|', encoding='ISO-8859-1', error_bad_lines=False)
df_BI_TEXT['FOI_TEXT'] = df_BI_TEXT['FOI_TEXT'].astype(str)
df_BI_TEXT = df_BI_TEXT.drop_duplicates('MDR_TEXT_KEY')
df_BI_TEXT['FOI_TEXT'] = df_BI_TEXT['FOI_TEXT'].map(processLine)
print(df_BI_TEXT.shape[0])
df_BI_TEXT['FOI_TEXT'].head(20)

27137


0     [healthcare, professional, reported, left, sid...
1     [device, evaluation, visual, analysis, returne...
2     [device, remains, implanted, review, device, h...
3     [patient, reported, left, side, collapsed, add...
4                  [deflation, resulting, explantation]
5                                                    []
6     [breast, augmentation, since, day, suffered, f...
7     [therapeutic, good, administration, reported, ...
8     [event, lymphoma, physiological, complication,...
9     [review, device, history, record, completed, d...
10    [healthcare, professional, reported, right, si...
11    [healthcare, professional, reported, left, sid...
12    [review, device, history, record, completed, d...
13    [device, received, analysis, time, submission,...
14    [reported, patient, suffered, bodily, injury, ...
15    [health, professional, reported, right, side, ...
16    [review, device, history, record, completed, d...
17    [article, citation, two, stage, implant, b

In [15]:
WL = []
for index, value in df_BI_TEXT.iterrows():
    WL += value['FOI_TEXT']
counter = Counter(WL)
with open("WF/Word_Cloud.txt", 'w') as f:
    for k,v in  counter.most_common():
        f.write( "{}\t{}\n".format(v,k) )

In [17]:
with open('WF/whole_text.txt', 'w') as f:
    for index, value in df_BI_TEXT.iterrows():
        for i in value['FOI_TEXT']:
            f.write(i + ' ')
        f.write('\n')